In [1]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

llm = ChatGroq(
    temperature=0,
    api_key=os.getenv('GROQ_API_KEY'),
    model='qwen-qwq-32b'
)

response = llm.invoke('Who is the first person to land on moon?')
print(response.content)


<think>
Okay, so I need to figure out who the first person to land on the moon was. Let me start by recalling what I know. I think it was someone from NASA's Apollo missions. I remember hearing about Neil Armstrong being the first, but I'm not entirely sure. Wait, wasn't there another astronaut with him on the same mission? Maybe Buzz Aldrin? Or was he the second one? Let me think.

I believe the Apollo 11 mission was the one that first landed on the moon. Neil Armstrong was the commander, right? So he would have been the first to step out of the lunar module. Then Buzz Aldrin followed him. But I should double-check the details to be certain. 

Wait, there's also Michael Collins, but he stayed in the command module orbiting the moon, so he didn't land. So the two who landed were Armstrong and Aldrin. The question is specifically about the first person. So Armstrong was the first to step onto the surface. 

I think the date was July 20, 1969. That's a date I've heard associated with th

In [2]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader('https://lifeattiktok.com/search/7258117514503506231')
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Solutions Engineer Manager, Monetization Technology#LifeAtTikTok#LifeAtTikTokDiversity & InclusionTeamsAdvertising & SalesCorporate FunctionsDesignE-CommerceGlobal OperationsMarketing & CommunicationsProductTechnologyTikTok  U.S. Data SecurityHow we hireApplying to TikTokInterview tipsFAQLocationsEarly CareersBlogJobsApplyCompanyAbout TikTokNewsroomContactProgramsTikTok for GoodTikTok for DevelopersEffect HouseAdvertise on TikTokTikTok RewardsTikTok BrowseTikTok EmbedsResourcesHelp centerSafety CenterCreator PortalCommunity GuidelinesTransparencyAccessibilityLegalPrivacy PolicyTerms of ServiceEnglish日本語TechnologySolutions Engineer Manager, Monetization TechnologyLocation:SingaporeEmployment Type:RegularJob Code:A204931Apply to this jobShare this listing:ResponsibilitiesTikTok is the leading destination for short-form mobile video. Our mission is to inspire creativity and bring joy. TikTok has global offices including Los Angeles, New York, London, Paris, Berlin, Dubai, Singapore, Jakar

In [3]:
import re
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    
    ### VALID JSON (NO PREAMBLE):
    """
)

def remove_thinking(text):
    """Remove content within <think> tags from the text."""
    return re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL).strip()

# Create a chain that processes the output to remove thinking
chain_extract = prompt_extract | llm | RunnableLambda(lambda x: remove_thinking(x.content))

result = chain_extract.invoke(input={'page_data': page_data})
print(result)

{
  "role": "Solutions Engineer Manager, Monetization Technology",
  "experience": "10+ years in programming/coding and 5+ years in technical management",
  "skills": [
    "Programming languages (Python, Go, Java, C/C++)",
    "Cross-functional collaboration",
    "Technical communication",
    "Advertising technology experience",
    "Data analytics"
  ],
  "description": "Manages the Solutions Engineering team to support APAC business by collaborating with cross-functional teams (R&D, Customer Success, Product Management). Responsibilities include leading engineering relationships with clients, driving joint roadmaps, identifying business opportunities, developing customized solutions, influencing product development, and leading both technical and people growth. The team focuses on enabling TikTok's advertising partners through technical solutions to achieve their goals."
}


In [4]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_result = json_parser.parse(result)
print(json_result)

{'role': 'Solutions Engineer Manager, Monetization Technology', 'experience': '10+ years in programming/coding and 5+ years in technical management', 'skills': ['Programming languages (Python, Go, Java, C/C++)', 'Cross-functional collaboration', 'Technical communication', 'Advertising technology experience', 'Data analytics'], 'description': "Manages the Solutions Engineering team to support APAC business by collaborating with cross-functional teams (R&D, Customer Success, Product Management). Responsibilities include leading engineering relationships with clients, driving joint roadmaps, identifying business opportunities, developing customized solutions, influencing product development, and leading both technical and people growth. The team focuses on enabling TikTok's advertising partners through technical solutions to achieve their goals."}


In [5]:
import polars

df = polars.read_csv('../data/my_portfolio.csv')
df = df.to_pandas()
df.head()

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio


In [6]:
import chromadb
from uuid import uuid4

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name='portfolio')

if not collection.count():
    for _, row in df.iterrows():
        collection.add(
            documents=row['Techstack'],
            metadatas={'links': row['Links']},
            ids=[str(uuid4())]
        )

In [7]:
links = collection.query(query_texts=json_result['skills'], n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/xamarin-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/typescript-frontend-portfolio'}],
 [{'links': 'https://example.com/android-tv-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}]]

In [8]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Frost, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating the seamless integration of business processes through automated tools. 
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, process optimization, cost reduction, and heightened overall efficiency. 
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase AtliQ portfolio: {link_list}
    Remember you are Frost, BDE at AtliQ. 
    Do not provide a preamble.
    
    ### EMAIL (NO PREAMBLE):
    """
)

chain_email = prompt_email | llm | RunnableLambda(lambda x: remove_thinking(x.content))
email_response = chain_email.invoke({'job_description': str(json_result), 'link_list': links})
print(email_response)

Subject: Strengthen APAC Monetization Strategy with AtliQ’s Technical Leadership  

Dear [Recipient's Name],  

As you drive TikTok’s APAC monetization goals, AtliQ’s team of senior engineers and data strategists can deliver the technical and collaborative expertise your team needs. Here’s how we align with your priorities:  

**Core Capabilities:**  
- **Programming & Data Mastery:** Expertise in Python, Java, and C/C++ for high-performance ad tech systems, paired with advanced data analytics to drive actionable insights.  
- **Ad Tech Integration:** Proven track record in building scalable advertising solutions, optimizing campaign performance, and aligning technical execution with business outcomes.  

**Cross-Functional Synergy:**  
- Seamless collaboration with R&D, product, and client teams to co-design joint roadmaps, identify monetization opportunities, and deliver client-centric solutions.  
- Influencing product development through data-driven strategies and technical innovat